## Tweets from old games

`scotrugbytweets` uses some rudimentary NLP-based named entity extraction from `nltk` with some fairly poor results. I've added a 'boring term' blacklist (team names, home nations, sponsors) but this is fairly unscalable and might miss out on some cool organic trends. In this notebook, I'll have a look at historical tweets around a Glasgow Warriors and Edinburgh Rugby game to get some insight into what is typically being tweeted about, and what might be interesting for `scotrugbytweets` visualisations.

In [17]:
import pandas as pd
from tweepy import OAuthHandler, Cursor, API
import yaml

#add directory above to path (listener Python namespace)
import sys
sys.path.append("..")

with open('../credentials.yaml', 'r') as f:
    creds = yaml.load(f)
    
auth = OAuthHandler(creds['consumer']['key'], creds['consumer']['secret'])
auth.set_access_token(creds['access']['key'], creds['access']['secret'])

Due to the week-limit on the Twitter API, we can only grab historical tweets from two Scottish games.

* **Glasgow Warriors** (A) v. Benetton Treviso (5/1/19 15:00KO)
* **Edinburgh** (H) v. Southern Kings (5/1/19 19:35 KO)

In [153]:
games_meta = {
    'GLA': {'kickoff': pd.to_datetime('2019-01-05 15:00'), 
            'twitter_punc': ['@GlasgowWarriors'], 
            'opposition': '@BenettonTreviso'},
    'EDI': {'kickoff': pd.to_datetime('2019-01-05 19:35'), 
            'twitter_punc': ['@EdinburghRugby'], 
            'opposition': '@SouthernKings'}
        }

def querify(hashtags):
    query = ' OR '.join(hashtags)
    return query

In [155]:
import csv
from nlp import clean_tweet, extract_entities
from itertools import chain

for team in games_meta.keys():
    print(team)
    csv_file = open('tweets_{0}.csv'.format(team), 'w')
    csv_writer = csv.writer(csv_file)
    
    #time definition
    window = '1 day'
    start_time = str(games_meta[team]['kickoff'] - pd.Timedelta(window)).split(' ')[0]
    end_time = str(games_meta[team]['kickoff'] + pd.Timedelta(window)).split(' ')[0]

    api = API(auth, wait_on_rate_limit=True)
    tweets = Cursor(api.search, q=querify(games_meta[team]['twitter_punc']), lang="en",
           since=start_time, until=end_time)

    for tweet in tweets.items():
        cleaned_tweet = clean_tweet(tweet.text)
        csv_writer.writerow([tweet.created_at, cleaned_tweet, extract_entities(cleaned_tweet)])

GLA
EDI


In [ ]:
tweets_df = pd.read_csv('./tweets_GLA.csv', header=None)
tweets_df.columns = ['timestamp', 'tweet', 'entities']

tweets_df['entities'] = tweets_df['entities'].apply(lambda row: row[1:-1].split(', '))
entities = pd.Series([row[1:-1] for row in list(chain(*tweets_df['entities'].values))])
entities = entities[entities != ''] #filter empty results

In [159]:
entities.value_counts()

BT                       94
TRY                      42
Great                    27
YOUR                     21
Big Bill                 20
GAME                     18
Hickey                   16
Fraser McKenzie          13
Darcy Graham             11
Fraz                     10
Try                      10
George Taylor            10
Darcy                    10
Simon                     9
Fowles                    9
Ross                      8
Cammy Fenton              8
MOTM                      8
Dougie Fife               8
Southern Kings            8
Ally Miller               8
Fife                      7
Simon Berghan             7
Ally                      7
Kings                     7
James                     7
Rory Sutherland           6
Charlie Shiel             6
Simon Hickey              6
Richard Cockerill         5
                         ..
Agreed                    1
First                     1
Charlie Shiels            1
Superb                    1
Wonder              